In [1]:
from astroquery.gaia import Gaia

import numpy as np
import pandas as pd
import sys
sys.path.append('../../scripts/')
from query import *

In [2]:
# Query parameters
TARGET_G_MAG_LIMIT = 6  # gs.phot_g_mean_mag < TARGET_G_MAG_LIMIT
NEIGHBOR_G_MAG_LIMIT = TARGET_G_MAG_LIMIT + 1  # neighbors.phot_g_mean_mag < NEIGHBOR_G_MAG_LIMIT
MIN_DEC = -85  # gs.dec >= MIN_DEC
MAX_DEC = 30  # gs.dec <= MAX_DEC
MIN_PARALLAX = 20  # gs.parallax >= MIN_PARALLAX (MAX_DISTANCE = 1000/MIN_PARALLAX)

In [3]:
query_all_stars = f"""
SELECT gs.source_id AS dr2_source_id,
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    gs.phot_bp_mean_mag, 
    gs.phot_rp_mean_mag, 
    gs.bp_rp, 
    gs.parallax
FROM gaiadr2.gaia_source AS gs
WHERE gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX} 
"""

no_bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag
FROM 
    gaiadr2.gaia_source AS gs
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND NOT EXISTS (
        SELECT 1
        FROM gaiadr2.gaia_source AS neighbors
        WHERE 
            1=CONTAINS(
                POINT('ICRS', gs.ra, gs.dec),
                CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
            )
            AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
            AND gs.source_id != neighbors.source_id
    )
"""

bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    neighbors.source_id AS neighbor_id,
    neighbors.phot_g_mean_mag AS neighbor_mag,
    DISTANCE(
        POINT('ICRS', gs.ra, gs.dec), 
        POINT('ICRS', neighbors.ra, neighbors.dec)
    ) AS angular_distance
FROM 
    gaiadr2.gaia_source AS gs
JOIN 
    gaiadr2.gaia_source AS neighbors
ON 
    1=CONTAINS(
        POINT('ICRS', gs.ra, gs.dec),
        CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
    )
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
    AND gs.source_id != neighbors.source_id
"""

# ... rest of the code remains the same ...

In [5]:


if 1: # For the all stars query
    df_all_stars = execute_gaia_query(
        query_all_stars,
        str_columns=['dr2_source_id'],
        output_file='all_stars_dr2.xlsx'
    )

if 1: # For the no bright neighbor query
    df_no_bright_neighbor = execute_gaia_query(
        no_bright_neighbor_query,
        str_columns=['source_id'],
        output_file='no_bright_neighbor_results.xlsx'
    )

if 1: # For the bright neighbor query
    df_bright_neighbor = execute_gaia_query(
        bright_neighbor_query,
        str_columns=['source_id', 'neighbor_id'],
        output_file='bright_neighbor_results.xlsx'
    )

INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 734
INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 717
INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 17


In [22]:
# merged_df = pd.read_excel('../../results/combined_query_with_mass_detection_limit.xlsx', dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})
merged_df = pd.read_excel('../../results/combined_query_with_mass_detection_limit.xlsx', dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str}).iloc[8:20]
merged_df

,source_id,source_id_dr2,source_id_dr3,RA,DEC,V_mag,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,...,Radius [R_Sun],HZ_limit [AU],RV precision [m/s],HZ Detection Limit [M_Earth],Spectral Type,HD Number,GJ Number,HIP Number,Object Type,HZ Detection Limit Simplified [M_Earth]
8,3903318372263850368,3903318372263850368,3903318372263850368,188.750913,9.827072,11.382888,10.452762,11.641875,9.360346,2.281529,...,0.501529,0.197819,0.504383,1.774141,M,NaN,GJ 476,61413,HighPM*,1.774131
9,4975284381905517824,4975284381905517824,NaN,11.436031,-47.548651,12.779940,11.892607,13.051886,10.249727,2.802158,...,NaN,1.025792,0.959914,NaN,M,HD 4391B,NaN,NaN,HighPM*,NaN
10,5970137861534914304,5970137861534914304,5970137861534914304,254.727158,-39.556574,10.480717,9.771798,10.732088,8.756553,1.975535,...,0.567853,0.250494,0.333617,1.383546,unknown,HD 153026B,GJ 646 B,NaN,HighPM*,1.383541
11,5946986063684477056,5946986063684477056,5946986063684477056,263.807237,-48.678777,10.221318,9.293614,10.480064,8.208224,2.271840,...,0.547341,0.183836,0.296742,0.954569,M,NaN,GJ 680,86057,HighPM*,0.954566
12,3828238392559860992,3828238392559860992,3828238392559860992,153.072937,-3.746748,9.233858,8.331761,9.491674,7.257185,2.234489,...,0.617193,0.223434,0.188832,0.631374,M,NaN,GJ 382,49986,RSCVnV*,0.631373
13,1450067137649449728,1450067137649449728,NaN,202.943071,29.276211,12.007520,10.607384,12.281963,9.379468,2.902495,...,0.440000,0.110073,0.677486,1.376924,NaN,NaN,NaN,NaN,NaN,1.376915
14,6164134146938406144,6164134146938406144,6164134146938406144,205.945200,-35.402677,11.771864,11.060765,12.022529,10.075360,1.947168,...,0.492493,0.198965,0.608386,2.146159,M,NaN,NaN,66993,HighPM*,2.146145
15,102648588798884736,102648588798884736,102648588798884736,36.366651,24.676904,12.192633,11.298914,12.450028,10.232467,2.217562,...,0.637770,0.265232,0.742001,3.229952,M,NaN,NaN,NaN,HighPM*,3.229925
16,689295212526177024,689295212526177024,689295212526177024,132.524693,23.531085,12.404436,11.661501,12.658576,10.571795,2.086782,...,0.657633,0.261231,0.820240,3.525215,M,NaN,NaN,NaN,LongPeriodV*,3.525182
17,6293916376311538944,6293916376311538944,6293916376311538944,208.221510,-18.338555,11.779842,10.892590,12.037050,9.826985,2.210065,...,0.473997,0.195173,0.615281,1.922747,M,NaN,NaN,67761,HighPM*,1.922734


In [28]:
def execute_gaia_query(query, str_columns=None, output_file=None, retries=3, delay=5):
    """
    Executes a Gaia query and optionally saves the results to an Excel file.

    Parameters:
    -----------
    query : str
        The ADQL query to execute.
    str_columns : list, optional
        List of column names to convert to string type.
    output_file : str, optional
        Path to save the Excel file. If None, no file is saved.
    retries : int
        Number of times to retry the query in case of failure.
    delay : int
        Delay in seconds between retries, with exponential backoff.

    Returns:
    --------
    pandas.DataFrame or None
        Query results as a DataFrame, or None if the query fails.
    """
    
    # Suppress the specific info message from astroquery
    import logging
    logging.getLogger('astroquery.utils.tap.core').setLevel(logging.WARNING)

    attempt = 0
    while attempt < retries:
        try:
            # Execute the query
            job = Gaia.launch_job_async(query)
            df = job.get_results().to_pandas()

            # Convert specified columns to string
            if str_columns:
                for col in str_columns:
                    if col in df.columns:
                        df[col] = df[col].astype(str)

            # Save to Excel if a filename is provided
            if output_file:
                df.to_excel(output_file, index=False)
                adjust_column_widths(output_file)

            # print(f"Number of results: {len(df)}")
            return df

        except requests.exceptions.HTTPError as e:
            print(f"An HTTP error occurred: {e}")
        except requests.exceptions.ConnectionError as e:
            print(f"A connection error occurred: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

        attempt += 1
        print(f"Retrying in {delay} seconds... (Attempt {attempt}/{retries})")
        time.sleep(delay)
        delay *= 2  # Exponential backoff

    print("Failed to execute query after several attempts.")
    return None


In [29]:
import pandas as pd
from astroquery.gaia import Gaia
import time

# Parameters
NEIGHBOR_G_MAG_LIMIT = 15  # Example brightness limit for neighbors
SEARCH_RADIUS = 2 / 3600.0  # 2 arcseconds in degrees

# Function to create a query for nearby stars
def create_neighbor_query_dr2(source_id, ra, dec, neighbor_g_mag_limit, search_radius):
    query = f"""
    SELECT 
        source_id, ra, dec, phot_g_mean_mag
    FROM 
        gaiadr2.gaia_source
    WHERE 
        1=CONTAINS(
            POINT('ICRS', {ra}, {dec}),
            CIRCLE('ICRS', ra, dec, {search_radius})
        )
        AND phot_g_mean_mag < {neighbor_g_mag_limit}
        AND source_id != {source_id}
    """
    return query

def create_neighbor_query_dr3(source_id, ra, dec, neighbor_g_mag_limit, search_radius):
    query = f"""
    SELECT 
        source_id, ra, dec, phot_g_mean_mag
    FROM 
        gaiadr3.gaia_source
    WHERE 
        1=CONTAINS(
            POINT('ICRS', {ra}, {dec}),
            CIRCLE('ICRS', ra, dec, {search_radius})
        )
        AND phot_g_mean_mag < {neighbor_g_mag_limit}
        AND source_id != {source_id}
    """
    return query

# DataFrames: merged_df (input sources)
rows_with_bright_neighbors = []
rows_without_bright_neighbors = []

# Iterate through each row in merged_df
for i, row in merged_df.iterrows():
    print(i, row['source_id_dr2'], row['source_id_dr3'])
    
    if not pd.isna(row['source_id_dr3']):
        query = create_neighbor_query_dr3(  
            source_id=row['source_id_dr3'],
            ra=row['RA'],
            dec=row['DEC'],
            neighbor_g_mag_limit=NEIGHBOR_G_MAG_LIMIT,
            search_radius=SEARCH_RADIUS
        )
    else:
        query = create_neighbor_query_dr2(
            source_id=row['source_id_dr2'],
            ra=row['RA'],
            dec=row['DEC'],
            neighbor_g_mag_limit=NEIGHBOR_G_MAG_LIMIT,
            search_radius=SEARCH_RADIUS
        )
    
    # Execute the query
    neighbors_df = execute_gaia_query(query)
    
    # Check if bright neighbors exist
    if neighbors_df is not None and not neighbors_df.empty:
        # Add the row to the bright neighbors list
        rows_with_bright_neighbors.append(row)
    else:
        # Add the row to the no bright neighbors list
        rows_without_bright_neighbors.append(row)

# Create DataFrames for rows with and without bright neighbors
bright_neighbors_df = pd.DataFrame(rows_with_bright_neighbors)
rows_without_bright_neighbors_df = pd.DataFrame(rows_without_bright_neighbors)

# Output the results
print(f"Rows with bright neighbors: {len(bright_neighbors_df)}")
print(f"Rows without bright neighbors: {len(rows_without_bright_neighbors_df)}")


8 3903318372263850368 3903318372263850368
INFO: Query finished. [astroquery.utils.tap.core]
9 4975284381905517824 nan
INFO: Query finished. [astroquery.utils.tap.core]
10 5970137861534914304 5970137861534914304
INFO: Query finished. [astroquery.utils.tap.core]
11 5946986063684477056 5946986063684477056
INFO: Query finished. [astroquery.utils.tap.core]
12 3828238392559860992 3828238392559860992
INFO: Query finished. [astroquery.utils.tap.core]
13 1450067137649449728 nan
INFO: Query finished. [astroquery.utils.tap.core]
14 6164134146938406144 6164134146938406144
INFO: Query finished. [astroquery.utils.tap.core]
15 102648588798884736 102648588798884736
INFO: Query finished. [astroquery.utils.tap.core]
16 689295212526177024 689295212526177024
INFO: Query finished. [astroquery.utils.tap.core]
17 6293916376311538944 6293916376311538944
INFO: Query finished. [astroquery.utils.tap.core]
18 3536123338748976256 3536123338748976256
INFO: Query finished. [astroquery.utils.tap.core]
19 422788884036

In [30]:
bright_neighbors_df


,source_id,source_id_dr2,source_id_dr3,RA,DEC,V_mag,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,...,Radius [R_Sun],HZ_limit [AU],RV precision [m/s],HZ Detection Limit [M_Earth],Spectral Type,HD Number,GJ Number,HIP Number,Object Type,HZ Detection Limit Simplified [M_Earth]
9,4975284381905517824,4975284381905517824,NaN,11.436031,-47.548651,12.77994,11.892607,13.051886,10.249727,2.802158,...,NaN,1.025792,0.959914,NaN,M,HD 4391B,NaN,NaN,HighPM*,NaN


In [33]:
output_path = 'bright_neighbors.xlsx'
bright_neighbors_df.to_excel(output_path, index=False)
adjust_column_widths(output_path)

